# 테스트 error를 최소화 하는 베타를 찾아보자.

In [1]:
import tensorflow as tf
import numpy as np

#### 데이터 생성

In [3]:
n = 300; p = 10
tf.compat.v1.random.set_random_seed(1234)
#b_vec = tf.constant([range(10,0,-1)])
#b_vec = tf.divide(b_vec,p).numpy()
x_mat = tf.random.normal([n,p]).numpy()
y_vec = tf.random.normal([n,1]).numpy()
#y_vec = np.matmul(x_mat,np.transpose(b_vec)) + tf.random.normal([n,1]).numpy()

#### 데이터 나누기

In [4]:
#train data
train_x_mat = x_mat[0:200,]
train_y_vec = y_vec[0:200]
test_x_mat = x_mat[200:301,]
test_y_vec = y_vec[200:301]

#### 텐서로 변환

In [5]:
#transform tf
train_x_mat = tf.constant(train_x_mat,dtype=tf.float32)
train_y_vec = tf.constant(train_y_vec,dtype=tf.float32)
test_x_mat = tf.constant(test_x_mat,dtype=tf.float32)
test_y_vec = tf.constant(test_y_vec,dtype=tf.float32)

#### W ,b 초기값 설정

In [6]:
# Weight and Bias: initialized randomly.
tf.compat.v1.random.set_random_seed(1)
W = tf.Variable(tf.random.normal([p, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

#### 선형회귀, 로스함수 정의

In [7]:
def linear_model(x):
    return tf.matmul(x,W) + b
    
# Mean square error.
def mean_square_loss(y_pred,y_true):
    return tf.reduce_mean(tf.pow(y_pred-y_true,2))

#### 학습 시키기 
- (200 epoch 해본 후, 135 epoch이 최적임을 알아냄)

In [8]:
#test로스를 최소화 하는 w,b 찾기
#epochs = 200
epochs = 135
batch_size = 5
learning_rate = 0.0005
optimizer = tf.optimizers.SGD(learning_rate)


train_data = tf.data.Dataset.from_tensor_slices((train_x_mat,train_y_vec))
training_batch = train_data.batch(batch_size).repeat(epochs)


# Interval for showing the training status.
display_step = len(train_y_vec)/batch_size

# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(training_batch, 1):
    # Run the optimization to update W and b values.
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = linear_model(batch_x)
        loss = mean_square_loss(pred, batch_y)

    # Compute gradients.
    trainable_variables = [W, b]
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    
    if step % display_step == 0:
        pred = linear_model(test_x_mat)
        loss = mean_square_loss(pred, test_y_vec)
        print("epoch: %i, loss: %.4f" % (step / display_step, loss))

epoch: 1, loss: 10.2703
epoch: 2, loss: 9.5241
epoch: 3, loss: 8.8400
epoch: 4, loss: 8.2129
epoch: 5, loss: 7.6376
epoch: 6, loss: 7.1099
epoch: 7, loss: 6.6256
epoch: 8, loss: 6.1811
epoch: 9, loss: 5.7730
epoch: 10, loss: 5.3982
epoch: 11, loss: 5.0538
epoch: 12, loss: 4.7374
epoch: 13, loss: 4.4466
epoch: 14, loss: 4.1792
epoch: 15, loss: 3.9332
epoch: 16, loss: 3.7070
epoch: 17, loss: 3.4989
epoch: 18, loss: 3.3073
epoch: 19, loss: 3.1310
epoch: 20, loss: 2.9686
epoch: 21, loss: 2.8190
epoch: 22, loss: 2.6811
epoch: 23, loss: 2.5541
epoch: 24, loss: 2.4371
epoch: 25, loss: 2.3291
epoch: 26, loss: 2.2296
epoch: 27, loss: 2.1377
epoch: 28, loss: 2.0530
epoch: 29, loss: 1.9748
epoch: 30, loss: 1.9026
epoch: 31, loss: 1.8360
epoch: 32, loss: 1.7744
epoch: 33, loss: 1.7175
epoch: 34, loss: 1.6650
epoch: 35, loss: 1.6165
epoch: 36, loss: 1.5716
epoch: 37, loss: 1.5301
epoch: 38, loss: 1.4917
epoch: 39, loss: 1.4562
epoch: 40, loss: 1.4234
epoch: 41, loss: 1.3930
epoch: 42, loss: 1.3649


#### 학습된 W와 b 확인

In [26]:
print('test error:',loss.numpy(),'\n')
print('b:',b.numpy(),'\n')
print('W:',W.numpy())

test error: 1.0078435 

b: [0.04962714] 

W: [[ 0.08158664]
 [-0.04445674]
 [ 0.03016776]
 [ 0.00207449]
 [ 0.02357498]
 [ 0.1417609 ]
 [ 0.000469  ]
 [-0.17816864]
 [ 0.05539262]
 [ 0.06723822]]


#### sklearn과 비교

In [25]:
from sklearn.linear_model import LinearRegression

# sklearn package
lrm = LinearRegression(n_jobs=-1)   
lrm.fit(train_x_mat,train_y_vec)
print("test error:",mean_square_loss(lrm.predict(test_x_mat),test_y_vec).numpy(),"\n")
print("b:",lrm.intercept_ ,"\n")
print("W:", lrm.coef_ )


test error: 1.0081856 

b: [0.04286529] 

W: [[ 0.08542245 -0.06130634  0.02489171  0.0090178   0.03340093  0.14455536
  -0.0048029  -0.18924932  0.06529539  0.06921971]]
